In [ ]:
from helpers import *
from RegularGrid import RegularGrid

### **Python file**

In [ ]:
category_name = '1D'

### **Rows**, **Columns** and more

In [ ]:
degree = 4
rows = 5
cols = 2
steps = 10
deployment= 'min'
folder_name ='00_test'

In [ ]:
folder_name , path_name = \
    create_dir_hierarchy(category_name,
                         degree,
                         rows,
                         cols,
                         deployment,
                         folder_name)
grid = RegularGrid(degree, rows, cols)

### **Deployed** and **Undeployed**

In [ ]:
deployed_cells  = [0,7]
undeployed_cells = grid.hline(1)

- grid.border()
- grid.center()
- grid.vline(line_idx)
- grid.hline(line_idx)
- grid.cross(arm_length)

---

In [ ]:
# extremities
active_cells    = deployed_cells + undeployed_cells
target_percents = [100]*len(deployed_cells) + [0]*len(undeployed_cells) # 0:undeployed | 100:deployed

In [ ]:
# write meta-data
with open(path_name+"/meta_data.txt", "w") as f:
    f.write("Degree: " + str(degree) + '\n')
    f.write("Rows  : " + str(rows)   + '\n')
    f.write("Cols  : " + str(cols)   + '\n')
    f.write("Deployment : " + str(deployment) + '\n')
    f.write("Steps      : " + str(steps)      + '\n')
    f.write("Active Cells    : " + str(active_cells)    + '\n')
    f.write("Target Percents : " + str(target_percents))

In [ ]:
grid.generate_mesh(folder_name)

In [ ]:
plot2D(grid.input_data, grid.curr_um, True, active_cells=active_cells, target_percents=target_percents, file_name = path_name+'/undeployed.png')

In [ ]:
view3D = plot3D(grid.curr_um, grid.input_data)
view3D.show()

In [ ]:
grid.deploy(active_cells, target_percents, view3D)

In [ ]:
if grid.rows==1 or grid.cols==1:
    file_name = path_name+'/projection2D.png'
    projection2D(grid.input_data, grid.curr_um, active_cells=active_cells, target_percents=target_percents, file_name=file_name)

In [ ]:
sview_vm = plot3D_stress(grid.curr_um, 'maxBending')
sview_vm.show()

In [ ]:
plot2D_stress(grid.curr_um, grid.input_data, grid.init_center_pos, active_cells, target_percents, show_percent=True)

### **Step Deployments**

In [ ]:
stresses_per_steps, percents_per_steps = \
deploy_in_steps(grid.curr_um,
                grid.input_data,
                grid.init_heights,
                grid.plate_thickness,
                active_cells,
                target_percents,
                steps=steps,
                stress_type='maxBending',
                verbose=True,
                dep=deployment)

### **CSV**

In [ ]:
# TODO
# -> to plot multiple curves/scatter plots on same figure

# arr = np.zeros((2,3))
# np.savetxt('./test.csv', arr, delimiter=',')
# arr_read = np.genfromtxt('./test.csv', delimiter=',')
# arr_read

### **images**

In [ ]:
plot2D_steps(grid.input_data,
             active_cells,
             percents_per_steps,
             grid.init_center_pos,
             stresses_per_steps,
             stress_type='maxBending',
             dir_name=path_name,
             show_percent=False,
             show_plot=False)

### **GIF**

In [ ]:
fps  = 3 # frames per seconds
loop = 2 # -1: no loop | 0: infinit loop | 1: loop once (see image twice) | 2: loop twice | ...
for name in ['structure_all', 'structure_perSteps', 'structure_own', 'scatter', 'sPlot']:
    img_name_i = f'"{path_name}/jpg/*{name}*.jpg"'
    gif_name_i = f'"{path_name}/jpg/gif/{name}.gif"'
    !ffmpeg  -loglevel panic -f image2 -r $fps -pattern_type glob -i $img_name_i -loop $loop $gif_name_i
    img_name_i = f'{path_name}/png/*{name}*.png'
    gif_name_i = f'{path_name}/png/gif/{name}.gif'
    !gifski --quiet -o $gif_name_i --fps $fps --repeat $loop $img_name_i
    